In [1]:
import pandas as pd

# these are new 
import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import shutil                   # file management tools 
import os          

In [49]:
my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

In [51]:
url = "https://api.census.gov/data/timeseries/intltrade/exports/hs?get=ALL_VAL_MO" + my_key+"&time=2013-01"

In [44]:
url

'https://api.census.gov/data/timeseries/intltrade/exports/hs?get=ALL_VAL_MO&key=34e40301bda77077e24c859c6c6c0b721ad73fc7&time=2013-01'

In [52]:
r = requests.get(url) 

In [53]:
r

<Response [200]>

In [54]:
r.json()

[['ALL_VAL_MO', 'time'], ['123031879061', '2013-01']]

https://api.census.gov/data/timeseries/intltrade/exports/naics?get=CTY_CODE,CTY_NAME,ALL_VAL_MO,ALL_VAL_YR&time=2013-05

In [198]:
end_use = "naics?get=CTY_CODE,CTY_NAME,ALL_VAL_MO,ALL_VAL_YR,CTY_NAME"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use + my_key + "&time=2019-01"

In [199]:
r = requests.get(url) 

In [70]:
naics_exports = pd.DataFrame(r.json())

In [72]:
naics_exports.head()

,0,1,2,3,4
0,CTY_CODE,CTY_NAME,ALL_VAL_MO,ALL_VAL_YR,time
1,-,TOTAL FOR ALL COUNTRIES,129545568405,129545568405,2019-01
2,0001,OPEC,4079752395,4079752395,2019-01
3,0003,EUROPEAN UNION,27836039515,27836039515,2019-01
4,0014,PACIFIC RIM COUNTRIES,28312301819,28312301819,2019-01


In [200]:
end_use = "naics?get=NAICS,CTY_CODE,COMM_LVL,ALL_VAL_MO,CTY_NAME"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use + my_key + "&time=2019-01" + "&COMM_LVL=NA3"

url = url + "&CTY_CODE=5700"

https://api.census.gov/data/timeseries/intltrade/exports/enduse?get=E_ENDUSE,ALL_VAL_MO,COMM_LVL&time=2013-01

In [201]:
url

'https://api.census.gov/data/timeseries/intltrade/exports/naics?get=NAICS,CTY_CODE,COMM_LVL,ALL_VAL_MO,CTY_NAME&key=34e40301bda77077e24c859c6c6c0b721ad73fc7&time=2019-01&COMM_LVL=NA3&CTY_CODE=5700'

In [202]:
r = requests.get(url) 


In [203]:
naics_exports = pd.DataFrame(r.json())

In [204]:
naics_exports.shape

(31, 8)

In [207]:
naics_exports.tail(31)

,0,1,2,3,4,5,6,7
0,NAICS,CTY_CODE,COMM_LVL,ALL_VAL_MO,CTY_NAME,time,COMM_LVL,CTY_CODE
1,111,5700,NA3,564521211,CHINA,2019-01,NA3,5700
2,112,5700,NA3,7622523,CHINA,2019-01,NA3,5700
3,113,5700,NA3,62684841,CHINA,2019-01,NA3,5700
4,114,5700,NA3,40059593,CHINA,2019-01,NA3,5700
5,211,5700,NA3,61525607,CHINA,2019-01,NA3,5700
6,212,5700,NA3,44544298,CHINA,2019-01,NA3,5700
7,311,5700,NA3,198960815,CHINA,2019-01,NA3,5700
8,312,5700,NA3,4598931,CHINA,2019-01,NA3,5700
9,313,5700,NA3,28923618,CHINA,2019-01,NA3,5700


In [171]:
test = "https://api.census.gov/data/timeseries/intltrade/exports/hs?get=CTY_CODE,CTY_NAME,ALL_VAL_MO&time=2013-01"

In [178]:
rtest  = requests.get(test) 

In [179]:
rtest.json()

[['CTY_CODE', 'CTY_NAME', 'ALL_VAL_MO', 'time'],
 ['-', 'TOTAL FOR ALL COUNTRIES', '123031879061', '2013-01'],
 ['1010', 'GREENLAND', '233446', '2013-01'],
 ['1220', 'CANADA', '23170845914', '2013-01'],
 ['2010', 'MEXICO', '17902453702', '2013-01'],
 ['2050', 'GUATEMALA', '425978783', '2013-01'],
 ['2080', 'BELIZE', '17795867', '2013-01'],
 ['2110', 'EL SALVADOR', '207606613', '2013-01'],
 ['2150', 'HONDURAS', '429806151', '2013-01'],
 ['2190', 'NICARAGUA', '75752432', '2013-01'],
 ['2230', 'COSTA RICA', '598484187', '2013-01'],
 ['2250', 'PANAMA', '1046236431', '2013-01'],
 ['2320', 'BERMUDA', '47156737', '2013-01'],
 ['2360', 'BAHAMAS', '256292297', '2013-01'],
 ['2390', 'CUBA', '65918611', '2013-01'],
 ['2410', 'JAMAICA', '199811433', '2013-01'],
 ['2430', 'TURKS AND CAICOS ISLANDS', '17119285', '2013-01'],
 ['2440', 'CAYMAN ISLANDS', '45294205', '2013-01'],
 ['2450', 'HAITI', '88937348', '2013-01'],
 ['2470', 'DOMINICAN REPUBLIC', '546345826', '2013-01'],
 ['2481', 'ANGUILLA', '197

In [180]:
total_month = pd.DataFrame(rtest.json())

In [185]:
total_month[total_month[1] == "CHINA"]

,0,1,2,3
142,5700,CHINA,9382923146,2013-01


In [183]:
total_month.head()

,0,1,2,3
0,CTY_CODE,CTY_NAME,ALL_VAL_MO,time
1,-,TOTAL FOR ALL COUNTRIES,123031879061,2013-01
2,1010,GREENLAND,233446,2013-01
3,1220,CANADA,23170845914,2013-01
4,2010,MEXICO,17902453702,2013-01


In [208]:
pd.append?

Object `pd.append` not found.
